In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [4]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/stop_clean.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [5]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'New|NNP',
 'The|DT',
 'after|IN',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'been|VBN',
 'be|VB',
 'but|CC',
 'by|IN',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'its|PRP$',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'the|DT',
 'this|DT',
 'to|IN',
 'to|TO',
 'two|CD',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'who|WP',
 'with|IN'}

In [6]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_eval_nopn.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(MSC1|NN, MSC2|NN)",20,-666.206003,2,1
1,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-666.206003,2,1
2,"(burial_place|VBP, burial_coordinates|NNS)",20,-670.238680,2,2
3,"(resting_place|NN, resting_place_coordinates|NNS)",20,-673.776236,2,2
4,"(Gm3|NN, ∕|SYM)",20,-677.905253,2,1
...,...,...,...,...,...
499995,"(and|CC, bit|NN)",31,-1837.108921,2,1
499996,"(also|RB, the|DT, day|NN)",22,-1837.109386,3,2
499997,"(public|NN, on|IN, a|DT)",23,-1837.110118,3,4
499998,"(coast|NN, during|IN, the|DT)",23,-1837.117093,3,7


In [7]:
min_freq = 20
eval_count = 500000

batch_count = max(ngram_eval.batch)

In [8]:
ngram_eval.batch.value_counts()

 1     171283
 2     122965
 3      76805
 4      51536
 5      30032
 6      19160
 7      12228
 8       6916
 9       3513
-1       3212
 10      2013
-2        337
Name: batch, dtype: int64

In [9]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [10]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [11]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/w10p_tagged_nopn_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 10
Loading w2v model
Gathering MWE stats
 MWE 0/171283: MSC1|NN+MSC2|NN
 MWE 5000/171283: dime|NN+store|NN
 MWE 10000/171283: expressing|VBG+support|NN
 MWE 15000/171283: writes|VBZ+songs|NNS
 MWE 20000/171283: Greatest|NNP+Adventure|NN
 MWE 25000/171283: later|RB+alleged|VBD
 MWE 30000/171283: and|CC+Javanese|JJ
 MWE 35000/171283: be|VB+formalized|VBN
 MWE 40000/171283: Nevada|NNP+as|IN
 MWE 45000/171283: 65th|JJ+Infantry|NNP
 MWE 50000/171283: and|CC+Negative|JJ
 MWE 55000/171283: initiated|VBN+under|IN
 MWE 60000/171283: her|PRP$+guidance|NN
 MWE 65000/171283: the|DT+Holkar|NNP
 MWE 70000/171283: two|CD+monuments|NNS
 MWE 75000/171283: the|DT+Thirty|NN-third|JJ
 MWE 80000/171283: He|PRP+penned|VBD
 MWE 85000/171283: with|IN+Theo|NNP
 MWE 90000/171283: State|NNP+1987|CD
 MWE 95000/171283: the|DT+Pulitzer|NNP+Prizes|NNPS
 MWE 100000/171283: and|CC+Viet|NNP+Cong|NNP
 MWE 105000/171283: misappropriation|NN+of|IN+funds|NNS
 MWE 110000/171283: the|DT+Edmonton|NNP+Pub

In [12]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_nopn_output_001.csv', index=False)

Appending batch 1 of 10
Appending batch 2 of 10
Appending batch 3 of 10
Appending batch 4 of 10
Appending batch 5 of 10
Appending batch 6 of 10
Appending batch 7 of 10
Appending batch 8 of 10
Appending batch 9 of 10
Appending batch 10 of 10


In [13]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(one|CD, on|IN, both|CC)",22,-1743.834796,3,4,"[1, 1, 0]","[-0.008623767, 0.02688746, -0.0819973, 0.04462...","[[-1.3924725, -0.32743967, -0.59011203, -1.932...","[nan, nan, -0.24756436]",-0.247564
1,"(were|VBD, once|IN, the|DT)",20,-1599.272787,3,7,"[1, 0, 1]","[0.021087596, -0.023043955, -0.13012876, 0.005...","[[-1.5887771, 0.37104017, 1.0021425, -3.168458...","[nan, -0.23794496, nan]",-0.237945
2,"(The|DT, four|CD, were|VBD)",21,-1718.246421,3,1,"[1, 0, 1]","[0.041595392, 0.02599036, -0.07892216, 0.11736...","[[-0.98371387, -1.1757944, 0.9406204, -1.07707...","[nan, -0.23560974, nan]",-0.235610
3,"(In|IN, addition|NN, their|PRP$)",20,-1565.757225,3,8,"[1, 0, 1]","[0.018744158, -0.027154934, -0.056828484, 0.13...","[[-0.81207585, 0.09480329, 0.8806889, -0.83343...","[nan, -0.227985, nan]",-0.227985
4,"(was|VBD, subsequently|RB, the|DT)",21,-1759.767362,3,7,"[1, 0, 1]","[0.10276595, 0.054750998, 0.0036850206, 0.0949...","[[-0.088278, 0.81340414, 0.77474403, -1.489990...","[nan, -0.22302704, nan]",-0.223027
...,...,...,...,...,...,...,...,...,...,...
499995,"(of|IN, I|PRP, 've|VBP)",23,-1786.381671,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499996,"((|-LRB-, BN|NN, –|SYM)",28,-1788.082423,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499997,"(/|SYM, 1,000|CD, live|JJ)",27,-1810.125560,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499998,"(/|,, vocals|NNS)",34,-1810.642550,2,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN


In [ ]:
#type(all_batches.stopwords[0])

In [14]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [15]:
w10p_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [16]:
w10p_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_nopn_light_001.csv', index=False)